## Linear Regression
- Select dataframe and target variable
- OLS
- Plot
- Shuffling

### Data

In [ ]:
# Load data
df_gm, df_meta = open_dataframes(path_df_gm, path_df_meta)

# Remove subjects with NaN values
target_col = "CDR_SB"
df_gm_regr = remove_missing_values(df_gm, df_meta, target_col)

# Features
df_merged, X = features_merging(df_gm_regr, df_meta)

# Print shapes
print("Old shape of the dataframe (data + ID): ", df_gm.shape)
print("New shape of the dataframe (data + ID): ", df_gm_regr.shape)
print("Shape of the complete dataframe (data + ID + metadata): ", df_merged.shape)
print("Shape of the metadata (ID included): ", df_meta.shape)
print("Shape of the input data X: ", X.shape)

### All subjects

In [ ]:
# UMAP
reducer = umap.UMAP(**umap_params)
X_umap = reducer.fit_transform(X)

# Target
y = df_merged[target_col]

# Add constant (intercept)
X_with_const = sm.add_constant(X_umap)

# Fit OLS
model = sm.OLS(y, X_with_const).fit()

# Predictions and residuals
y_pred = model.predict(X_with_const)
residuals = y - y_pred

# Summary
print(model.summary())

#### Plotting

In [ ]:
plot_ols_diagnostics(y, y_pred, residuals, "OLS Actuals vs Predicted and Residuals",save_path=None, plot_flag=True, color_by_group=True, group_labels=df_merged['Group'])

In [ ]:
#plot_actual_vs_predicted(target = y, predictions=  y_pred, "OLS Distributions", save_path=None, plot_flag=True):

#### Covariates

In [ ]:
# UMAP
reducer = umap.UMAP(**umap_params)
X_umap = reducer.fit_transform(X)

# Target
y = df_merged[target_col].astype(float)

# Covariate encoding
covariates = ["Sex", "Age", "Education"]
df_cov = df_merged[covariates].copy()

# Ensure numeric types
df_cov["Age"] = pd.to_numeric(df_cov["Age"], errors="coerce")
df_cov["Education"] = pd.to_numeric(df_cov["Education"], errors="coerce")
df_cov = pd.get_dummies(df_cov, drop_first=True)

# Combine UMAP + covariates
df_umap = pd.DataFrame(X_umap, columns=["UMAP1", "UMAP2"])
X_design = pd.concat([df_umap.reset_index(drop=True), df_cov.reset_index(drop=True)], axis=1)

# Final casting
X_design = X_design.astype(float)

# Add constant
X_with_const = sm.add_constant(X_design)

# Fit OLS
model = sm.OLS(y, X_with_const).fit()

# Predictions and residuals
y_pred = model.predict(X_with_const)
residuals = y - y_pred

# Summary
print(model.summary())

#### Shuffling Regression

In [ ]:
# R² real
r2_real = model.rsquared

# List of R² after shuffling
r2_shuffled = []

for _ in range(100):
    y_shuffled = y.sample(frac=1, replace=False, random_state=None).reset_index(drop=True)
    model_shuffled = sm.OLS(y_shuffled, X_with_const).fit()
    r2_shuffled.append(model_shuffled.rsquared)

# empiric p-value
p_value = np.mean([r >= r2_real for r in r2_shuffled])

print(f"R² real: {r2_real:.4f}")
print(f"R² shuffled: {np.mean(r2_shuffled):.4f}")
print(f"p-value (shuffling): {p_value:.4f}")

### GMM distributions

In [ ]:
# Load data
df_gm, df_meta = open_dataframes(path_df_gm, path_df_meta)

In [ ]:
# CDR means
for gmmLabel in range(0,3):
    mean_cdr = df_meta.loc[df_meta['labels_gmm_cdr'] == gmmLabel, 'CDR_SB'].mean()
    print(f"Media CDR_SB per gruppo {gmmLabel}: {mean_cdr:.3f}")

In [ ]:
target_col = "CDR_SB"
label_col = "labels_gmm_cdr"

# Loop over each GMM cluster
for gmm_label in df_meta[label_col].dropna().unique():
    print(f"\n=== GMM Cluster {gmm_label} ===")

    # Get IDs for the current cluster
    ids_in_cluster = df_meta[df_meta[label_col] == gmm_label]["ID"]

    # Filter metadata and remove missing target values
    df_meta_cluster = df_meta[df_meta["ID"].isin(ids_in_cluster)].reset_index(drop=True)
    df_meta_cluster = df_meta_cluster[df_meta_cluster[target_col].notna()]

    # Filter voxel data accordingly
    df_cluster = df_gm[df_gm["ID"].isin(df_meta_cluster["ID"])].reset_index(drop=True)

    # Run regression
    model, y_pred, residuals, subject_errors, group_rmse_stats = main_regression(
        df_masked = df_cluster,
        df_meta = df_meta_cluster,
        target_variable = target_col,
        covariates = ["Sex", "Age", "Education"],
        y_log_transform = False,
        plot_flag = False,
        save_path = None,
        title_prefix = f"gm_mask_GMM_{gmm_label}"
    )

### Cluster K-Means

In [ ]:
# Load data
df_gm, df_meta = open_dataframes(path_df_gm, path_df_meta)

In [ ]:
target_col = "CDR_SB"
cluster_col = "KMeans"

for cluster_id in labels_df[cluster_col].unique():
    print(f"\n=== Cluster {cluster_id} ===")

    # Get subject IDs for the current cluster
    ids_in_cluster = labels_df[labels_df[cluster_col] == cluster_id]["ID"]

    # Filter df_meta and df_gm_masked based on IDs only
    df_meta_cluster = df_meta[df_meta["ID"].isin(ids_in_cluster)].reset_index(drop=True)
    df_cluster = df_gm[df_gm["ID"].isin(ids_in_cluster)].reset_index(drop=True)

    # Run regression (NaN removal is handled internally)
    model, y_pred, residuals, subject_errors, group_rmse_stats = main_regression(
        df_masked = df_cluster,
        df_meta = df_meta_cluster,
        target_variable = target_col,
        covariates = ['Sex', 'Age', 'Education'],
        y_log_transform = False,
        plot_flag = True,
        save_path = None,
        title_prefix = f"cluster_{cluster_col}_{cluster_id}"
    )